### Import the libraries

In [3]:
!pip install "numpy<2.0" --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:

!sudo apt update
!sudo apt install -y build-essential cmake ninja-build patchelf autoconf libtool pkg-config

# reinstall sklearn
!pip install numpy==1.24 scikit-learn --no-binary :all:


Hit:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:2 https://packages.microsoft.com/repos/azure-cli jammy InRelease           
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:4 http://archive.lambdalabs.com/ubuntu jammy InRelease                     
Hit:5 https://download.docker.com/linux/ubuntu jammy InRelease                 
Hit:6 https://pkg.cloudflare.com/cloudflared jammy InRelease                   
Hit:7 http://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease    
Hit:8 https://ll-cloud-repo.s3.us-west-2.amazonaws.com jammy InRelease0m       
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease              
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:11 https://ppa.launchpadcontent.net/flexiondotorg/nvtop/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main am

In [1]:

# standard libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import scanpy as sc
from scipy.sparse import csr_matrix


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### Import the data

In [ ]:

adata_100m=sc.read_h5ad("/home/ubuntu/anatoly-tahoe-100-texas/data/tahoe-100m_5M.h5ad")
